## Download and extract the dataset
---
### Download link expires on October 24, 2019

In [1]:
!rm -r *
!wget https://transfer.sh/fcpg1/dataset.zip
!unzip -q dataset_80_10_10.zip

--2019-12-08 07:23:50--  https://transfer.sh/fcpg1/dataset.zip
Resolving transfer.sh (transfer.sh)... 144.76.136.153
Connecting to transfer.sh (transfer.sh)|144.76.136.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 454498692 (433M) [application/x-zip-compressed]
Saving to: ‘dataset.zip’

dataset.zip         100%[===================>] 433.44M  29.1MB/s    in 16s     

2019-12-08 07:24:08 (26.7 MB/s) - ‘dataset.zip’ saved [454498692/454498692]

unzip:  cannot find or open dataset_80_10_10.zip, dataset_80_10_10.zip.zip or dataset_80_10_10.zip.ZIP.


In [0]:
working_dir =   "/content"
train_dir   =   "/content/dataset/train/"
val_dir     =   "/content/dataset/val/"
test_dir    =   "/content/dataset/test/"

# আসল খেলা

In [2]:
from keras.callbacks import *
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16, Xception, MobileNetV2

Using TensorFlow backend.


In [0]:
filepath = working_dir + "/xception.hdf5"

checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
csv_logger = CSVLogger('xception_training.log')
early_stopping = EarlyStopping(monitor="acc", patience=2)
callbacks_list = [checkpoint, csv_logger, early_stopping]

# Transfer Learning VGG16


In [5]:
# vgg = VGG16(include_top=False, weights='imagenet', input_shape=(32,32, 3))
# for layer in vgg.layers[:]:
#     layer.trainable = False

# vgg = VGG16(include_top=False, input_shape=(32,32, 3))
# model = Sequential()
# model.add(vgg)

xc = Xception(include_top=False, input_shape=(71,71, 3), weights='imagenet')
model = Sequential()
model.add(xc)

# mb = MobileNetV2(include_top=False, input_shape=(28,28, 3), weights='imagenet')
# model = Sequential()
# model.add(mb)


model.add(Flatten())
model.add(Dropout(0.25))
model.add(Dense(122, activation='softmax'))

sgd = optimizers.SGD(lr=0.1)

model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])













Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




In [6]:
batch_size = 32

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=False)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(71, 71),
        batch_size=batch_size,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        val_dir,
        target_size=(71, 71),
        batch_size=batch_size,
        class_mode='categorical')

Found 290135 images belonging to 122 classes.
Found 36223 images belonging to 122 classes.


In [7]:
model.fit_generator(
        train_generator,
        epochs=20,
        validation_data=validation_generator,
        callbacks=callbacks_list,
        verbose = 1)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/20
9067/9067 [==============================] - 737s 81ms/step - loss: 0.4274 - acc: 0.8901 - val_loss: 0.1945 - val_acc: 0.9493

Epoch 00001: val_acc improved from -inf to 0.94929, saving model to /content/xception.hdf5
Epoch 2/20
9067/9067 [==============================] - 722s 80ms/step - loss: 0.1733 - acc: 0.9539 - val_loss: 0.1513 - val_acc: 0.9605

Epoch 00002: val_acc improved from 0.94929 to 0.96052, saving model to /content/xception.hdf5
Epoch 3/20
9067/9067 [==============================] - 732s 81ms/step - loss: 0.1339 - acc: 0.9642 - val_loss: 0.1505 - val_acc: 0.9610

Epoch 00003: val_acc improved from 0.96052 to 0.96102, saving model to /content/xception.hdf5
Epoch 4/20
9067/9067 [==============================] - 733s 81ms/step - loss: 0.1112 - acc: 0.9699 - val_loss: 0.1452 - val_acc: 0.9636

Epoch 00004: val_acc improved from 0.96102 to 0.96361, saving model to /c

In [0]:
model = load_model("xception.hdf5")
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(71, 71),
        batch_size=batch_size,
        class_mode='categorical')
model.evaluate_generator(test_generator)

Found 36374 images belonging to 122 classes.


[0.15999225848512377, 0.9599714081388456]

In [0]:
from google.colab import files
files.download('training.log') 